In [57]:
import pickle
import os
import sys
sys.path.append("../")
from od.social.group import SocialGroup
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS,NET_TIMEOUT_SECONDS
from od.env.station import BS_PRESET
import od.misc.statistic as oms

In [134]:
print(os.getcwd())
statistics = []
for i,loc in enumerate(["TVAL_CONST_1s","TVAL_CONST_300ms"]):
    interest_config = InterestConfig(True,ResourceAllocatorType.NOMA_APR,True,1.3,7)
    with open("../data/{}/{}/object.pickle".format(loc,interest_config.folder()), "rb") as file:
        statistics.append(pickle.load(file))
        statistics[-1].nft_traffic = {}

/home/OD/repos/ProjectSumo/ipynb


In [135]:
statistics

In [136]:

BS_CONFIG =  {
        name:bs_config 
        for name,bs_config in BS_PRESET.items()
        if (
            bs_config["type"] == BaseStationType.UMA  or 
            (
                interest_config.req_rsu and 
                bs_config["type"] == BaseStationType.UMI
            )
        )
}

class FakeBaseStation:
    def __init__(self,name,pos,bs_type,serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial
    def __repr__(self) -> str:
        return self.name
    def __index__(self):
        return self.serial
        
class GV:
    NET_STATION_CONTROLLER = [ 
        FakeBaseStation(
            name,
            BS_CONFIG[name]["pos"],
            BS_CONFIG[name]["type"] ,
            i
        )   for i,name in enumerate(BS_CONFIG)
    ]
    class STATISTIC:
        def Log(data):
            pass
        def Doc(data):
            pass
    class RESULT:
        def Log(data):
            pass
        def Doc(data):
            print(data)
oms.GV = GV
def RecordPrinter(header):
    print("time at:{}".format(header.at))
    print("bs ot:{}".format(header.is_bs_ot))
    print("src ot:{}".format(header.is_src_ot))
    print("time bs drop:{}".format(header.time_bs_drop))
    print("time bs serv:{}".format(header.time_bs_serv))
    print("time bs tx:{}".format(header.time_bs_tx))
    print("time bs txq:{}".format(header.time_bs_txq))
    print("time bs trip:{}".format(header.time_veh_trip))

In [137]:
GV.NET_STATION_CONTROLLER

[UMA-1, UMA-2, UMI-1, UMI-2, UMI-3, UMI-4, UMI-5]

In [138]:
for statistic in statistics:
    for sg in statistic.social_group:
        for record in statistic.sg_header[sg].values():
            for bs in GV.NET_STATION_CONTROLLER:
                if(len(record.time_bs_txq[bs])==0):
                    continue
                if(record.time_bs_txq[bs][-1][1] - record.at >= NET_TIMEOUT_SECONDS):
                    record.is_bs_ot[bs] = True

In [139]:
for i,statistic in enumerate(statistics):
    print("====={}======".format(i))
    for sg in statistic.social_group:
        ot_count = 0 
        for header, record in statistic.sg_header[sg].items():
            if record.is_src_ot > 0:
                ot_count +=1
            elif (True in [record.is_bs_ot[i] for i in GV.NET_STATION_CONTROLLER]):
                ot_count +=1

        print("{}: {} => {}".format(sg,ot_count, ot_count / max(len(statistic.sg_header[sg]),1)))
            

=====0======
CRASH: 0 => 0.0
RCWS: 0 => 0.0
DYN_SG_0: 0 => 0.0
DYN_SG_1: 0 => 0.0
DYN_SG_2: 0 => 0.0
DYN_SG_3: 1127 => 0.7101449275362319
DYN_SG_4: 0 => 0.0
DYN_SG_5: 0 => 0.0
DYN_SG_6: 0 => 0.0
DYN_SG_7: 0 => 0.0
DYN_SG_8: 189 => 1.0
=====1======
CRASH: 0 => 0.0
RCWS: 0 => 0.0
DYN_SG_0: 0 => 0.0
DYN_SG_1: 0 => 0.0
DYN_SG_2: 0 => 0.0
DYN_SG_3: 1133 => 0.7139256458727158
DYN_SG_4: 0 => 0.0
DYN_SG_5: 0 => 0.0
DYN_SG_6: 0 => 0.0
DYN_SG_7: 0 => 0.0
DYN_SG_8: 189 => 1.0


In [140]:
class Helper:
    def __init__(self,header,record,bs_index):
        self.name = header
        self.bits = record.bits
        self.time_gen = round(record.at,4)
        self.time_serv = round(record.time_bs_serv[bs_index],4)
        self.time_len =  195-185 if self.time_serv == -1 else round(self.time_serv - self.time_gen,4)
    def __str__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)
    def __repr__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)


In [141]:
target_header = "flow_2.34-117"
# target_header = "flow_0.58-99"
for statistic in statistics:
    header_in_sg_list = [sg.fname for sg in statistic.social_group   if(target_header in statistic.sg_header[sg])]
    print("In {}".format(header_in_sg_list))
    print("C?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL)))
    print("G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.GENERAL)))
    print("C2G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.C2G)))

In ['CRASH', 'RCWS']
C?:True
G?:True
C2G?:True
In ['CRASH', 'RCWS']
C?:True
G?:True
C2G?:True


In [142]:
for i,statistic in enumerate(statistics):
    for sg in statistic.social_group:
        if target_header in statistic.sg_header[sg]:
            header = statistic.sg_header[sg][target_header]
            print("======={}:{}========".format(i,sg))
            RecordPrinter(header)

=======0:CRASH========
time at:186.439
bs ot:[False, False, False, False, False, False, False]
src ot:False
time bs drop:[-1, -1, -1, -1, -1, -1, -1]
time bs serv:[-1, -1, -1, 186.443, -1, -1, -1]
time bs tx:[[], [], [], [[186.443, 186.4435]], [], [], []]
time bs txq:[[], [], [], [[186.443, 186.443]], [], [], []]
time bs trip:{'flow_0.37': 0.004500000000007276, 'flow_0.38': 0.004500000000007276, 'flow_0.39': 0.004500000000007276, 'flow_0.40': 0.004500000000007276, 'flow_0.41': 0.004500000000007276, 'flow_0.42': 0.004500000000007276, 'flow_0.43': 0.004500000000007276, 'flow_0.44': 0.004500000000007276, 'flow_0.45': 0.004500000000007276, 'flow_0.46': 0.004500000000007276, 'flow_0.47': 0.004500000000007276, 'flow_0.48': 0.004500000000007276, 'flow_1.7': 0.004500000000007276, 'flow_1.8': 0.004500000000007276, 'flow_13.3': 0.004500000000007276, 'flow_13.4': 0.004500000000007276, 'flow_2.28': 0.004500000000007276, 'flow_2.29': 0.004500000000007276, 'flow_2.31': 0.004500000000007276, 'flow_2.

In [143]:
bs_index = 3
social_groups = statistics[0].social_group
print(social_groups)
# app_stats_fn = lambda x: statistic.sg_header[social_groups[3]]
app_stats_fn = lambda x: statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL)
# app_stats_fn = lambda x: x.sg_header[SocialGroup.RCWS]

[<CRASH(QoS:0, Gid:0)>, <RCWS(QoS:1, Gid:1)>, <DYN_SG_0(QoS:1, Gid:2)>, <DYN_SG_1(QoS:1, Gid:3)>, <DYN_SG_2(QoS:1, Gid:4)>, <DYN_SG_3(QoS:1, Gid:5)>, <DYN_SG_4(QoS:1, Gid:6)>, <DYN_SG_5(QoS:1, Gid:7)>, <DYN_SG_6(QoS:1, Gid:8)>, <DYN_SG_7(QoS:1, Gid:9)>, <DYN_SG_8(QoS:1, Gid:10)>]


In [144]:
bs_serv_list = [[] for _ in statistics]
bs_time_gap = [{} for _ in statistics]
for i, statistic in enumerate(statistics):
    for header, record in app_stats_fn(statistic).items():
        if(len(record.time_bs_txq[bs_index]) > 0):
            bs_serv_list[i].append(Helper(header, record, bs_index))
    bs_serv_list[i].sort(key=lambda x : x.time_gen*1000000+x.time_serv)
    for helper in sorted(bs_serv_list[i],key=lambda x: (x.time_serv * 1000) * 10000000 + x.bits):
        at = str(helper.time_gen)
        to = str(helper.time_serv)
        if(to not in bs_time_gap[i]):
             bs_time_gap[i][to] = {"need": 0, "give": 0}
        if(at not in bs_time_gap[i]):
             bs_time_gap[i][at] = {"need": 0, "give": 0}
        bs_time_gap[i][to]["give"]+=helper.bits
        bs_time_gap[i][at]["need"]+=helper.bits
    
   


In [145]:
for v in bs_serv_list[0]:
    print(v)

flow_0.46-64[ at:186.0 ,to:186.001, dur:0.001, size:2696]
flow_0.41-65[ at:186.001 ,to:186.002, dur:0.001, size:6824]
flow_0.47-64[ at:186.001 ,to:186.002, dur:0.001, size:8760]
flow_1.7-64[ at:186.002 ,to:186.003, dur:0.001, size:2720]
flow_1.8-65[ at:186.002 ,to:186.003, dur:0.001, size:5560]
flow_2.29-65[ at:186.002 ,to:186.003, dur:0.001, size:4288]
flow_2.39-64[ at:186.002 ,to:186.003, dur:0.001, size:3736]
flow_0.39-64[ at:186.002 ,to:186.004, dur:0.002, size:6648]
flow_0.43-65[ at:186.002 ,to:186.004, dur:0.002, size:6952]
flow_0.45-64[ at:186.002 ,to:186.004, dur:0.002, size:7776]
flow_0.38-64[ at:186.003 ,to:186.004, dur:0.001, size:2600]
flow_2.32-64[ at:186.003 ,to:186.005, dur:0.002, size:4136]
flow_2.33-64[ at:186.003 ,to:186.005, dur:0.002, size:6192]
flow_2.36-65[ at:186.003 ,to:186.005, dur:0.002, size:6272]
flow_2.34-65[ at:186.003 ,to:186.006, dur:0.003, size:8552]
flow_2.35-65[ at:186.003 ,to:186.006, dur:0.003, size:6656]
flow_4.1-64[ at:186.003 ,to:186.006, dur:0.0

In [146]:
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    data = []
    for i in range(2):
        if(time in bs_time_gap[i]):
            data.append(bs_time_gap[i][time]["need"])
            data.append(bs_time_gap[i][time]["give"])
        else:
            data.append(0)
            data.append(0)
    if(not sum(data)==0):
        print(time, data)


186.0 [2696, 0, 2696, 0]
186.001 [15584, 2696, 15584, 2696]
186.002 [37680, 15584, 37680, 15584]
186.003 [40840, 16304, 40840, 16304]
186.004 [38568, 23976, 38568, 23976]
186.005 [15896, 16600, 15896, 16600]
186.006 [11160, 24624, 11160, 24624]
186.007 [0, 19720, 0, 19720]
186.008 [0, 18744, 0, 18744]
186.009 [0, 17464, 0, 17464]
186.01 [0, 6712, 0, 6712]
186.016 [4912, 0, 4912, 0]
186.017 [5544, 4912, 5544, 4912]
186.018 [35168, 5544, 35168, 5544]
186.019 [17264, 20464, 17264, 20464]
186.02 [45800, 19784, 45800, 19784]
186.021 [19856, 20280, 19856, 20280]
186.022 [31280, 21120, 31280, 21120]
186.023 [9696, 19888, 9696, 19888]
186.024 [0, 20576, 0, 20576]
186.025 [0, 18560, 0, 18560]
186.026 [0, 18392, 0, 18392]
186.033 [8040, 0, 8040, 0]
186.034 [20952, 8040, 20952, 8040]
186.035 [15112, 14320, 15112, 14320]
186.036 [23216, 21744, 23216, 21744]
186.037 [44352, 15760, 44352, 15760]
186.038 [5656, 22568, 5656, 22568]
186.039 [28840, 21664, 28840, 21664]
186.04 [13864, 17048, 13864, 1704

In [147]:

remain = [0 for _ in statistics]
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    for i in range(2):
        if(time in bs_time_gap[i]):
            helper = bs_time_gap[i][time]
            remain[i] += (helper["need"] - helper["give"])
    if(not sum(remain)==0):
        print(time, remain, remain[0]-remain[1])


186.0 [2696, 2696] 0
186.001 [15584, 15584] 0
186.002 [37680, 37680] 0
186.003 [62216, 62216] 0
186.004 [76808, 76808] 0
186.005 [76104, 76104] 0
186.006 [62640, 62640] 0
186.007 [42920, 42920] 0
186.008 [24176, 24176] 0
186.009 [6712, 6712] 0
186.016 [4912, 4912] 0
186.017 [5544, 5544] 0
186.018 [35168, 35168] 0
186.019 [31968, 31968] 0
186.02 [57984, 57984] 0
186.021 [57560, 57560] 0
186.022 [67720, 67720] 0
186.023 [57528, 57528] 0
186.024 [36952, 36952] 0
186.025 [18392, 18392] 0
186.033 [8040, 8040] 0
186.034 [20952, 20952] 0
186.035 [21744, 21744] 0
186.036 [23216, 23216] 0
186.037 [51808, 51808] 0
186.038 [34896, 34896] 0
186.039 [42072, 42072] 0
186.04 [38888, 38888] 0
186.041 [20640, 20640] 0
186.05 [11608, 11608] 0
186.051 [39184, 39184] 0
186.052 [42552, 42552] 0
186.053 [37912, 37912] 0
186.054 [51856, 51856] 0
186.055 [44840, 44840] 0
186.056 [43744, 43744] 0
186.057 [28456, 28456] 0
186.058 [10008, 10008] 0
186.066 [9736, 9736] 0
186.068 [36208, 36208] 0
186.069 [66336, 6